# Preprocessing

## Import

In [ ]:
import os
import pathlib
import matplotlib

import numpy as np
import matplotlib.pyplot as plt
import mne
import mne_bids
from mne.preprocessing import (ICA, create_eog_epochs, 
                               compute_proj_eog, create_ecg_epochs, corrmap) 

matplotlib.use('Agg') #Raises no browsers
#matplotlib.use('Qt5Agg') #Raises browsers
mne.set_log_level('warning') #All mne functions will only spit out output if they have reason to.

## Directory

In [ ]:
eeg_path = "eeg"
subname = "sub-"+subject
sesname = "ses-"+session
epoch_reports = "02_epochs"
preprocessed_reports = "03_preprocessed"

In [ ]:
#Makes EEG_OUTPUTS -> subject -> session -> eeg -> preprocessed_reports
if not os.path.exists(os.path.join(output_path, subname, sesname, 
                                   eeg_path, preprocessed_reports)):
    os.makedirs(os.path.join(output_path, subname, sesname, 
                             eeg_path, preprocessed_reports))

In [ ]:
epoch_report_path = os.path.join(root_path, "EGI_OUTPUTS\\"+subname+"\\"+sesname+"\\"+eeg_path+"\\"+epoch_reports)

In [ ]:
preprocessed_report_path = os.path.join(root_path, "EGI_OUTPUTS\\"+subname+"\\"+sesname+"\\"+eeg_path+"\\"+preprocessed_reports)

## ICA Analysis

#### Input Data

In [ ]:
bids_path = mne_bids.BIDSPath(subject=subject,
                              session=session,
                              task=task,
                              datatype='eeg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path) 
events, event_id = mne.events_from_annotations(raw) 
raw.load_data() 

#### Interpolate Bads

In [ ]:
raw.info['bads'] = bad_list

In [ ]:
raw = raw.copy().interpolate_bads(reset_bads=True)

#### Reset Channel Types: Input

In [ ]:
raw = raw.copy().pick_types(meg=False, eeg=True, eog=True)
raw.set_channel_types({'E8': 'eog'})
raw.set_channel_types({'E25': 'eog'})
len(raw.ch_names)
raw

In [ ]:
raw.set_eeg_reference('average',ch_type='eeg',projection=True)  # needed for inverse modeling

#### Pass Data Filter

In [ ]:
high_pass_raw = raw.filter(l_freq=1, h_freq=None) 

#### Data ICA Analysis

In [ ]:
ica = ICA(n_components=15, max_iter='auto', random_state=97)
ica.fit(high_pass_raw)
ica

In [ ]:
ica.plot_components()

#### ICA Report

In [ ]:
ica_copy = ica.copy()

In [ ]:
print('######################## Computing ICA analysis. ########################')

In [ ]:
eog_epochs = mne.preprocessing.create_eog_epochs(raw=raw)
eog_components, eog_scores = ica_copy.find_bads_eog(
    inst=eog_epochs,
    ch_name=['E8','E25'],  # a channel close to the eye
    threshold=1  # lower than the default threshold
)
ica_copy.exclude = eog_components

In [ ]:
ica_report_title = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_ica_report.html'
ica_report = mne.Report(title='ICA Filter')
ica_report.add_ica(
    ica=ica_copy,
    title='ICA cleaning',
    picks=None,  
    inst=raw,
    eog_evoked=eog_epochs.average(),
    eog_scores=eog_scores,
    n_jobs=1  # could be increased!
)
ica_report_path = os.path.join(preprocessed_report_path, ica_report_title)
ica_report.save(ica_report_path, overwrite=True)